In [1]:
# https://github.com/groq/groq-api-cookbook/blob/main/tutorials/structured-output-instructor/structured_output_instructor.ipynb

In [2]:
import os
import instructor
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from groq import Groq
from pprint import pprint

model = "llama-3.3-70b-versatile"

# Load the Groq API key from .env file
load_dotenv()
GROQ_API_KEY = os.environ.get("GROQ_API_KEY")
print(f"GROQ API Key exists and begins {GROQ_API_KEY[:14]}...")

GROQ API Key exists and begins gsk_11hFN1EMfj...


In [3]:
# Describe the desired output schema using pydantic models
# Pydantic is a coercion and validation library and we will ber able to gracefully handle the output knowing its structure.
class UserInfo(BaseModel):
    name: str
    age: int
    email: str


# The text to extract data from
text = """
John Doe, a 35-year-old software engineer from New York, has been working with large language models for several years.
His email address is johndoe@example.com.
"""

# Patch Groq() with instructor, this is where the magic happens!
client = instructor.from_groq(Groq(), mode=instructor.Mode.JSON)

# Call the API
user_info = client.chat.completions.create(
    model=model,
    response_model=UserInfo,  # Specify the response model
    messages=[
        {
            "role": "system",
            "content": "Your job is to extract user information from the given text.",
        },
        {"role": "user", "content": text},
    ],
    temperature=0.65,
)
print(type(user_info))
print(f"Name: {user_info.name}")
print(f"Age: {user_info.age}")
print(f"Email: {user_info.email}")

<class '__main__.UserInfo'>
Name: John Doe
Age: 35
Email: johndoe@example.com


In the example above, we've defined a simple pydantic model `UserInfo` that specifies a person's name (as a string), age (as an integer), and email (as a string). The `instructor` library ensures that the Groq model's output adheres to this schema. The great thing here is that the `instructor` library ensures the response is valid according to the schema you provided. This eliminates the need for manual validation and reduces the likelihood of errors creeping into your data.